In [3]:
class merkeltreecommit:
    def __init__(self, mesg_byte = 24 , mesg_len = 2**16 , hash_byte = 32 , leaf_padding_byte = 32 ):
        self.hash_byte = hash_byte
        self.leaf_padding_byte = leaf_padding_byte
        self.mesg_byte = mesg_byte
        assert( 0==(mesg_len&(mesg_len-1)) )   # mesg_len is a power of 2
        self.mesg_len   = mesg_len

    @staticmethod
    def authpath_len( leaf_size ):
        r = 0
        layer_size = leaf_size
        while(layer_size > 1):
            r += 1
            layer_size //= 2
        return r
    
    @property
    def len_authpath(self): return merkeltreecommit.authpath_len(self.mesg_len)
 
    def get_size_root(self): return self.hash_byte

    def get_size_leaf(self): return self.mesg_byte + self.leaf_padding_byte

    def get_size_proof(self, num_query=1):
        r = self.get_size_root()
        r += num_query * self.get_size_leaf() 
        r += num_query * merkeltreecommit.authpath_len( self.mesg_len ) * self.hash_byte
        return r
    
    def states(self):
        r = vars(self)
        r['authpath_len'] = merkeltreecommit.authpath_len(self.mesg_len)
        r['proof_size']  = self.get_size_proof()
        return r


In [4]:
mc0 = merkeltreecommit()
print( "|auth path|:" , mc0.len_authpath )
print( "size(auth path):" , mc0.get_size_proof() )
print( mc0.states() )

|auth path|: 16
size(auth path): 600
{'hash_byte': 32, 'leaf_padding_byte': 32, 'mesg_byte': 24, 'mesg_len': 65536, 'authpath_len': 16, 'proof_size': 600}


In [10]:
class fri :

    # polys:  [ f_w , f_Az , f_Bz , f_Cz , f_r , f_rLDT ]  , [ f_h ]
    def __init__(self, field_byte = 24 , hash_byte = 32 , poly_len = 2**14 , num_polys = [ 6 ,1 ], code_rate = 32 , num_query = 26 ):
        self.field_byte = field_byte
        self.hash_byte  = hash_byte
        assert( 0==(poly_len&(poly_len-1)) )   # poly_len is a power of 2
        self.poly_len   = poly_len
        self.num_polys = num_polys
        self.code_rate  = code_rate
        self.num_query  = num_query

    def get_size_proof(self):
        r = 0
        plen = self.poly_len
        # first layer
        for np in self.num_polys :
           mt0 = merkeltreecommit( mesg_byte=self.field_byte*np , mesg_len = plen*self.code_rate , hash_byte=self.hash_byte , leaf_padding_byte = 32 )
           r += mt0.get_size_proof( self.num_query )
        plen //= 2
        # i-th layer
        while( plen > 1 ):
            mt = merkeltreecommit( mesg_byte=self.field_byte , mesg_len = plen*self.code_rate , hash_byte=self.hash_byte , leaf_padding_byte = 32 )
            r += mt.get_size_proof( self.num_query )
            plen //= 2
        # plen == 1
        r += self.field_byte
        return r
    
    def states(self):
        r = vars(self)
        r['proof_size']  = self.get_size_proof()
        return r

In [11]:
pc0 = fri()
print( pc0.get_size_proof() )
print( pc0.states() )

186872
{'field_byte': 24, 'hash_byte': 32, 'poly_len': 16384, 'num_polys': [6, 1], 'code_rate': 32, 'num_query': 26, 'proof_size': 186872}


In [20]:
class ligero :
    # polys:  [ f_w , f_Az , f_Bz , f_Cz , f_r ]  , [ f_h , f_g ]
    def __init__(self, field_byte = 24 , hash_byte = 32 , poly_len = 2**14 , num_polys = [ 5 ,2 ],  num_segments = 1 , code_rate = 32 , num_query = 26 ):
        self.field_byte = field_byte
        self.hash_byte  = hash_byte
        assert( 0==(poly_len&(poly_len-1)) )   # poly_len is a power of 2
        self.poly_len   = poly_len
        self.num_polys = num_polys
        self.num_segments = num_segments
        self.code_rate  = code_rate
        self.num_query  = num_query

    def get_size_proof(self):
        r = 0
        p_seg_len = self.poly_len // self.num_segments
        # first layer
        for np in self.num_polys :
           mt0 = merkeltreecommit( mesg_byte=self.field_byte*self.num_segments*np , mesg_len = p_seg_len*self.code_rate , hash_byte=self.hash_byte , leaf_padding_byte = 32 )
           r += p_seg_len * self.field_byte
           r += mt0.get_size_proof( self.num_query )
        return r
    
    def states(self):
        r = vars(self)
        r['proof_size']  = self.get_size_proof()
        return r

In [21]:
pc1 = ligero()
print( pc1.get_size_proof() )
print( pc1.states() )

824144
{'field_byte': 24, 'hash_byte': 32, 'poly_len': 16384, 'num_polys': [5, 2], 'num_segments': 1, 'code_rate': 32, 'num_query': 26, 'proof_size': 824144}


In [22]:
for i in range(13):
    num_segments = 1<<i
    pc = ligero( field_byte=24, hash_byte=32, poly_len=2**14 , num_polys = [5,2] , num_segments=num_segments , code_rate=32, num_query=26 )
    print( pc.states() )

{'field_byte': 24, 'hash_byte': 32, 'poly_len': 16384, 'num_polys': [5, 2], 'num_segments': 1, 'code_rate': 32, 'num_query': 26, 'proof_size': 824144}
{'field_byte': 24, 'hash_byte': 32, 'poly_len': 16384, 'num_polys': [5, 2], 'num_segments': 2, 'code_rate': 32, 'num_query': 26, 'proof_size': 433632}
{'field_byte': 24, 'hash_byte': 32, 'poly_len': 16384, 'num_polys': [5, 2], 'num_segments': 4, 'code_rate': 32, 'num_query': 26, 'proof_size': 244096}
{'field_byte': 24, 'hash_byte': 32, 'poly_len': 16384, 'num_polys': [5, 2], 'num_segments': 8, 'code_rate': 32, 'num_query': 26, 'proof_size': 161600}
{'field_byte': 24, 'hash_byte': 32, 'poly_len': 16384, 'num_polys': [5, 2], 'num_segments': 16, 'code_rate': 32, 'num_query': 26, 'proof_size': 145728}
{'field_byte': 24, 'hash_byte': 32, 'poly_len': 16384, 'num_polys': [5, 2], 'num_segments': 32, 'code_rate': 32, 'num_query': 26, 'proof_size': 189376}
{'field_byte': 24, 'hash_byte': 32, 'poly_len': 16384, 'num_polys': [5, 2], 'num_segments': 